# Cleaning Data Spreadsheet
This notebook breaks down the process for importing data into pandas dataframes, cleaning the data, and preprocessing it.

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy import stats
from datetime import datetime
from functools import reduce
import datetime 
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
import statsmodels.api as sm

Importing the Datasets that have been downloaded from Yahoo Finance, as well as Dukascopy.com (Currencies)

In [2]:
snp = pd.read_csv('Datasets/Snp500.csv')
gld = pd.read_csv('Datasets/SPDR_Gold.csv')
eur = pd.read_csv('Datasets/EURUSD.csv')
usd = pd.read_csv('Datasets/USDJPY.csv')
cbnd = pd.read_csv('Datasets/Corp_Bonds.csv')
tbnd = pd.read_csv('Datasets/7_10_yr_Treasury.csv')

The Currency Data accounts for weekend prices (in which case the final price on Friday evening is considered as the Sat/Sun price as the markets are closed). Thus I filter to remove weekend prices (i.e. where trade volume is 0). I also describe the shapes of the datasets in order to grasp the size (in days) of the data.

In [3]:
#currencies are counting for weekends
#remove dates where volume is 0
eur['Local time'] = eur['Local time'].shift(-1)
usd['Local time'] = usd['Local time'].shift(-1)
eur = eur[eur.Volume != 0]
usd = usd[usd.Volume != 0]
dflist = [snp,gld,eur,usd,cbnd,tbnd]
for i, dfr in enumerate(dflist):
    print(dflist[i].shape)
#SNP is from 1980 (fix to include only since 2003)

(9910, 7)
(3629, 7)
(4163, 6)
(4161, 6)
(4211, 7)
(4211, 7)


In [4]:
#Leaving only Date Close Price and Volume
snp = snp.drop(['Open','High','Low','Adj Close'], axis = 1)
gld = gld.drop(['Open','High','Low','Adj Close'], axis = 1)
cbnd = cbnd.drop(['Open','High','Low','Adj Close'], axis = 1)
tbnd = tbnd.drop(['Open','High','Low','Adj Close'], axis = 1)
eur = eur.drop(['Open','High','Low'], axis = 1)
usd = usd.drop(['Open','High','Low'], axis = 1)

In [5]:
#Changing first column to datetime ans simplifying (only day, not time)
snp['Date'] = pd.to_datetime(snp['Date']).dt.date
gld['Date'] = pd.to_datetime(gld['Date']).dt.date
cbnd['Date'] = pd.to_datetime(cbnd['Date']).dt.date
tbnd['Date'] = pd.to_datetime(tbnd['Date']).dt.date
eur.rename(columns = {'Local time':'Date'}, inplace = True)
usd.rename(columns = {'Local time':'Date'}, inplace = True)
eur['Date'] = pd.to_datetime(eur['Date'], dayfirst = True).dt.date
usd['Date'] = pd.to_datetime(usd['Date'], dayfirst = True).dt.date

In [6]:
#Renaming Columns that will be fed into the larger dataframe
snp.rename(columns = {'Close':'SNP_Close','Volume':'SNP_Volume'}, inplace = True)
gld.rename(columns = {'Close':'GLD_Close','Volume':'GLD_Volume'}, inplace = True)
eur.rename(columns = {'Close':'EUR_Close','Volume':'EUR_Volume'}, inplace = True)
usd.rename(columns = {'Close':'USD_Close','Volume':'USD_Volume'}, inplace = True)
cbnd.rename(columns = {'Close':'CBND_Close','Volume':'CBND_Volume'}, inplace = True)
tbnd.rename(columns = {'Close':'TBND_Close','Volume':'TBND_Volume'}, inplace = True)

Here I index and sort each asset class by the date column.

Then, I merge the individual asset classes by their common index (so as to make sure that I will be fairly training the models on datasets encompassing the same date range, and in order to make storing the asset classes easier.

In [7]:
dflist = [snp,gld,eur,usd,cbnd,tbnd]
for i, dfr in enumerate(dflist):
    dflist[i].set_index('Date',inplace = True)
    dflist[i].sort_index(inplace = True)

from functools import reduce
dflist = [snp,gld,eur,usd,cbnd,tbnd]
df_final = reduce(lambda left, right: pd.merge(left, right, on = 'Date'),dflist)

In [8]:
df_final.to_csv('Final_Data_Prices.csv')

This is the main preprocessing stage where I perform two different transformations of the data.

For each price column, I calculate the logged returns for each date as the Log(Price at time t, Price at time t-1). This is a common technique used in analyzing time series data, and it has many advantages in model calculation stemming from log properties on small changes (i.e. for small r, log(1+r) is approximately r). 

For each volume column, I normalize the volumes. The information from the volumes in magnitude and scale is not as important as the values relative to their mean (i.e. how extreme is the volume relative to the norm). In doing so, I assume that the volume has a constant mean that is independent of time and constant variance. In reality, it is likely that volume is more like prices and this may not neccesarily be true. I may need to tweak this in the future.

In [8]:
j = 0
for i in df_final:
    if j % 2 == 0:
        df_final[i] = np.log(df_final[i]/df_final[i].shift(1))
    else:
        df_final[i] = preprocessing.scale(df_final[i].values)
    j+=1

C:\Users\alexk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\alexk\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\Users\alexk\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\alexk\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  war

In [9]:
#Dropping the first row as it is NaN as a result of the Log Calculations
df_final.drop(df_final.index[0],inplace = True)
print(df_final.shape)
df_final.head()

(3627, 12)


,SNP_Close,SNP_Volume,GLD_Close,GLD_Volume,EUR_Close,EUR_Volume,USD_Close,USD_Volume,CBND_Close,CBND_Volume,TBND_Close,TBND_Volume
Date,,,,,,,,,,,,
2004-11-19,-0.011224,-1.740063,0.008973,0.271874,0.004888,1.418665,-0.010727,0.082009,-0.005258,-0.819043,-0.005495,0.489545
2004-11-22,0.005878,-1.848352,0.003789,0.318723,0.001956,1.396135,0.000378,0.080116,-0.000447,-0.793952,0.000703,-0.608626
2004-11-23,-0.000255,-1.819561,-0.004459,-0.895031,0.003007,1.380100,0.001598,0.080919,0.001251,-0.838895,-0.000938,-0.600090
2004-11-24,0.004087,-2.044953,0.006682,-0.491322,0.007309,1.392799,-0.005153,0.084316,-0.000089,-0.843064,0.001406,-0.672622
2004-11-26,0.000753,-2.566598,0.005313,-0.904862,0.008452,1.351151,-0.002114,0.075066,-0.003578,-0.852473,-0.003637,-0.666729


In [10]:
#Exporting the DataSet
df_final.to_csv('Final_Data.csv')

In [9]:
#Testing Data Set Cleanup
btc_data = pd.read_csv('BTC-USD.csv')

In [10]:
btc_data = btc_data.drop(['Open','High','Low','Adj Close'], axis = 1)
btc_data['Date'] = pd.to_datetime(btc_data['Date']).dt.date
btc_data.rename(columns = {'Close':'BTC_Close','Volume':'BTC_Volume'}, inplace = True)
btc_data.set_index('Date',inplace = True)
btc_data.sort_index(inplace = True)
btc_data.to_csv('BTC_Prices.csv')

In [ ]:
btc_data['BTC_Close'] = np.log(btc_data['BTC_Close']/btc_data['BTC_Close'].shift(1))
btc_data['BTC_Volume'] = preprocessing.scale(btc_data['BTC_Volume'].values)
btc_data.drop(btc_data.index[0],inplace = True)
print(btc_data.shape)
btc_data.head()

In [13]:
btc_data.to_csv('BTC_Final.csv')